# Data enrichment

In [85]:
# Importing libraries
import os
from rdkit import Chem
from rdkit.Chem import Descriptors
import pandas as pd
import requests
from dotenv import dotenv_values

## 1. Chemical descriptors

Now, let's obtain the chemicals descriptos. For this, [RDKit](https://github.com/rdkit/rdkit) collection of cheminformatics will be used.

In [2]:
def rdkit_descriptors():
    '''
    This is a function for getting the list of all molecular descriptors in RDKit package
    '''

    # List of attributes to drop
    Methods_exception = [
                         '_FingerprintDensity',
                         '_isCallable', '_runDoctests',
                         '_setupDescriptors',
                         'setupAUTOCorrDescriptors',
                         '_ChargeDescriptors'
                         ]

    # Getting list of attributes as functions
    methods =  {func: getattr(Descriptors, func) for func in dir(Descriptors)
                if type(getattr(Descriptors, func)).__name__ == "function"
                and func not in Methods_exception}
    methods = {s: methods[s] for s in sorted(methods)}

    return methods

In [3]:
def descriptors_for_chemical(SMILES):
    '''
    This is a function for collecting all the descriptor for a molecule
    '''

    descriptors = None

    # Molecule from SMILES
    molecule = Chem.MolFromSmiles(SMILES)

    if molecule is not None:
        # Molecular descriptors
        descriptors = {}
        for descriptor_name, descriptor_func in rdkit_descriptors().items():
            try:
                descriptors.update({descriptor_name: [descriptor_func(molecule)]})
            except ZeroDivisionError:
                descriptors.update({descriptor_name: None})

    return descriptors

In [4]:
def information_for_set_of_chems(
                                 col_id,
                                 df_chems
                                 ):
    '''
    This is a function to look for the descriptors for all molecules
    '''    

    # Iterating over the dataframe rows (chemicals)
    df_descriptors = pd.DataFrame()
    for _, row in df_chems.iterrows():
        descriptors = descriptors_for_chemical(row['smiles'])
        if descriptors is None:
            continue
        else:
            descriptors.update({col_id: row[col_id]})
            df_descriptors = \
                pd.concat([df_descriptors,
                        pd.DataFrame(descriptors)])
            del descriptors

    # Merging descriptors and input parameters
    df_chems = pd.merge(df_descriptors,
                        df_chems,
                        how='right',
                        on=col_id)
    del df_descriptors

    return df_chems

In [5]:
# Opening the dataset

df = pd.read_csv(os.path.join(os.getcwd(),
            os.pardir,
            'data',
            'transformed',
            'dataset_after_eda.csv'))

In [6]:
# Drop records without SMILES

df = df[pd.notnull(df['smiles'])]

In [7]:
df.head()

,source_reduction_general_category,description_code,2_digit_naics,smiles,cas_number,reporting_year
0,Good Operating Practices,"greater than or equal 5%, but less than to 15%","Mining, Quarrying, and Oil and Gas Extraction",C=O,50-00-0,2014
1,Good Operating Practices,"greater than or equal 5%, but less than to 15%","Mining, Quarrying, and Oil and Gas Extraction",C(C(CO[N+](=O)[O-])O[N+](=O)[O-])O[N+](=O)[O-],55-63-0,2014
2,Good Operating Practices,"greater than or equal 5%, but less than to 15%","Mining, Quarrying, and Oil and Gas Extraction",C1=CC=C(C=C1)C2(C(=O)NC(=O)N2)C3=CC=CC=C3,57-41-0,2014
3,Good Operating Practices,"greater than or equal 5%, but less than to 15%","Mining, Quarrying, and Oil and Gas Extraction",C1=CC=C(C=C1)N,62-53-3,2014
4,Good Operating Practices,"greater than or equal 5%, but less than to 15%","Mining, Quarrying, and Oil and Gas Extraction",COP(=O)(OC)OC=C(Cl)Cl,62-73-7,2014


In [8]:
# Organizing unique chemicals

df_chem = df[['smiles', 'cas_number']].drop_duplicates(keep='first').reset_index(drop=True)
df.drop(['smiles'], axis=1, inplace=True)

In [9]:
# Searching for chemical descriptors

df_chem = information_for_set_of_chems('cas_number', df_chem)

In [10]:
# Merging both datasets

df = pd.merge(df, df_chem, on='cas_number', how='inner')

In [11]:
# Dropping columns that are no longer required

df1 = df.drop(['smiles', 'cas_number', 'reporting_year'], axis=1)

In [12]:
# Saving the result dataset

df1.to_csv(os.path.join(os.getcwd(),
                        os.pardir,
                        'data',
                        'transformed',
                        'dataset_after_enrichment.csv.zip'),
         index=False, 
         compression='zip')

## 2. Chemical prices

Let's use the prices from Scifinder for the chemicals.

In [13]:
df.drop(['smiles'], axis=1, inplace=True)

In [70]:
df_prices = pd.read_csv(os.path.join(os.getcwd(),
                        os.pardir,
                        'scripts',
                        'data_acquisition',
                        'chem_prices.csv'))

In [15]:
df_prices.head()

,price_usd_g,generic_substance_id
0,0.22,78933
1,0.16,64175
2,1.32,NA-08
3,0.11,71432
4,0.52,NA-10


In [16]:
df_prices.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 643 entries, 0 to 642
Data columns (total 2 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   price_usd_g           643 non-null    float64
 1   generic_substance_id  643 non-null    object 
dtypes: float64(1), object(1)
memory usage: 10.2+ KB


In [17]:
df_prices.rename(columns={'generic_substance_id': 'cas_number'}, inplace=True)

In [18]:
df_chem['cas_number'] = df_chem['cas_number'].str.replace('-', '')
df['cas_number'] = df['cas_number'].str.replace('-', '')

In [19]:
df_chem = pd.merge(df_chem, df_prices, on='cas_number', how='left')

In [20]:
df_chem.loc[df_chem.price_usd_g.isnull(), ['cas_number', 'price_usd_g']]

,cas_number,price_usd_g
20,75025,NaN
138,22248799,NaN
182,8021394,NaN
185,12185103,NaN


In [21]:
# Lazy and fast imputation using median

df_chem.loc[df_chem.price_usd_g.isnull(), 'price_usd_g'] = df_chem.price_usd_g.median()

In [22]:
df = pd.merge(df, df_chem[['cas_number', 'price_usd_g']], on='cas_number', how='left')

In [23]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 965300 entries, 0 to 965299
Columns: 406 entries, source_reduction_general_category to price_usd_g
dtypes: float64(297), int64(105), object(4)
memory usage: 2.9+ GB


In [101]:
# As Public Administration (not covered in economic census) is not in census, it must be removed

df = df[df['2_digit_naics'] != 'Public Administration (not covered in economic census)']

## 3. Gross value added

Let's know using information from the U.S. Bureau of Economic Analysis (BEA) to obtain the Gross Domestic Product by Industry.

In [102]:
naics_dict = {
    'Agriculture, Forestry, Fishing and Hunting (not covered in economic census)': '11',
    'Mining, Quarrying, and Oil and Gas Extraction': '21',
    'Utilities': '22',
    'Manufacturing': '31ND',
    'Wholesale Trade': '42',
    'Transportation and Warehousing': '48TW',
    'Information': '51',
    'Professional, Scientific, and Technical Services': '54',
    'Administrative and Support and Waste Management and Remediation Services': '56',
    'Educational Services': '61',
    'Health Care and Social Assistance': '62',
    'Other Services (except Public Administration)': '81',
}

In [124]:
def getting_gpd(Year, Industry, UserID):
    '''
    Function to request the GPD by Industry Sector
    '''
    
    url = f'https://apps.bea.gov/api/data/?&UserID={UserID}&method=GetData&DataSetName=GDPbyIndustry&Frequency=A&Industry={Industry}&TableID=1&Year={Year}'

    response = requests.get(url)
    
    if response.status_code == 200:
        
        return float(response.json()['BEAAPI']['Results'][0]['Data'][0]['DataValue'])
        
    else:
        
        return None
        raise ValueError(f'Error: {response.status_code}')
        

In [116]:
# Loading the UserID for using BEA info

config = dotenv_values(os.path.join(os.getcwd(),
                        os.pardir,
                        '.env'))

USERID = config['USERID']

In [125]:
df_industry = df[['2_digit_naics', 'reporting_year']].drop_duplicates(keep='first')

In [126]:
# Searching for GDP by Industry

df_industry['gdp'] = df_industry.apply(lambda row: getting_gpd(row['reporting_year'],
                                         naics_dict[row['2_digit_naics']],
                                         USERID),
                  axis=1)

In [127]:
df_industry.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 46 entries, 0 to 4890
Data columns (total 3 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   2_digit_naics   46 non-null     object 
 1   reporting_year  46 non-null     int64  
 2   gdp             46 non-null     float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.4+ KB


In [128]:
df = pd.merge(df, df_industry, on=['2_digit_naics', 'reporting_year'], how='left')

In [129]:
df.drop(['reporting_year', 'cas_number'], axis=1, inplace=True)

In [ ]:
# Saving the result dataset

df.to_csv(os.path.join(os.getcwd(),
                        os.pardir,
                        'data',
                        'transformed',
                        'dataset_after_enrichment_economic_vals.csv.zip'),
         index=False, 
         compression='zip')